# Heavy Hexagon model and examples from Kim et al., Nature, 618, 500 (2023).
## Reproduces results of Fig 3 of Begusic, Gray, Chan, Sci. Adv. 10, eadk4321 (2024). 
Reference data available at https://github.com/tbegusic/arxiv-2308.05077-data.git in file spd_10.csv.

In [ ]:
import sys
sys.path.append('../')
from spd.extras.HeavyHexUtils import *
from spd.OperatorSequence import *
from spd.SparsePauliDynamics import *
from qiskit.quantum_info import PauliList
import matplotlib.pyplot as plt
from copy import deepcopy
import pickle

In [ ]:
nq = 127
n_zz = 144
coupling_map = coupling_list()

### Perform Clifford and angle transformation (only once and store resulting observables and circuits)
Next four cells can be commented out after Clifford transformed information is stored.

In [ ]:
#5-step examples of Fig 3a-c
n_layers = 5
#Magnetization, weight-10 and weight-17 observables.
observables = [
    PauliList([pauli_op([i], nq, 'Z') for i in range(nq)]), 
    PauliList([pauli_op_XYZ([[13, 29, 31], [9, 30], [8, 12, 17, 28, 32]], nq)]), 
    PauliList([pauli_op_XYZ([[37, 41, 52, 56, 57, 58, 62, 79], [75], [38, 40, 42, 63, 72, 80, 90, 91]], nq)]) 
    ]

#For theta_h < pi/4 (transform with ZZ Cliffords and keep only non-Clifford operators)
op_seq_prep = op_seq_layers(n_layers, coupling_map, 0.0, nq, n_zz)
op_seq_prep.reduce_clifford()
ops_small_angle_5steps = extract_non_clifford(op_seq_prep)
observables_small_angle_5steps = [op_seq_prep.clifford_ops.apply_to_PauliList(obs) for obs in observables]

#For theta_h > pi/4 (in addition, do the angle transformation to transform all angles back to )
op_seq_prep = OperatorSequence(deepcopy(ops_small_angle_5steps), [[1.5*np.pi/8]*nq]*n_layers) #Using a dummy coefficient > pi/8 to invoke an angle transformation.
op_seq_prep.reduce_clifford()
ops_large_angle_5steps = op_seq_prep.ops
observables_large_angle_5steps = [op_seq_prep.clifford_ops.apply_to_PauliList(obs) for obs in observables_small_angle_5steps]

In [ ]:
#5+1-step example of Fig 3d
n_layers = 5
observable = PauliList([pauli_op_XYZ([[37, 41, 52, 56, 57, 58, 62, 79], [38, 40, 42, 63, 72, 80, 90, 91], [75]], nq)]) 

#For theta_h < pi/4
op_seq_prep = op_seq_layers_3d(n_layers, coupling_map, 0.0, nq, n_zz)
op_seq_prep.reduce_clifford()
ops_small_angle_3d = extract_non_clifford_3d(op_seq_prep)
observable_small_angle_3d = op_seq_prep.clifford_ops.apply_to_PauliList(observable)
#For theta_h > pi/4
op_seq_prep = OperatorSequence(deepcopy(ops_small_angle_3d), [[1.5*np.pi/8]*nq]*(n_layers+1)) #Using a dummy coefficient > pi/8 to invoke an angle transformation.
op_seq_prep.reduce_clifford()
ops_large_angle_3d = op_seq_prep.ops
observable_large_angle_3d = op_seq_prep.clifford_ops.apply_to_PauliList(observable_small_angle_3d)

In [ ]:
#20-step example of Fig 3e
n_layers = 20
observable = PauliList([pauli_op_XYZ([[], [], [62]], nq)])

#For theta_h < pi/4
op_seq_prep = op_seq_layers(n_layers, coupling_map, 0.0, nq, n_zz)
op_seq_prep.reduce_clifford()
ops_small_angle_3e = extract_non_clifford(op_seq_prep)
observable_small_angle_3e = op_seq_prep.clifford_ops.apply_to_PauliList(observable)
#For theta_h > pi/4
op_seq_prep = OperatorSequence(deepcopy(ops_small_angle_3e), [[1.5*np.pi/8]*nq]*n_layers) #Using a dummy coefficient > pi/8 to invoke an angle transformation.
op_seq_prep.reduce_clifford()
ops_large_angle_3e = op_seq_prep.ops
observable_large_angle_3e = op_seq_prep.clifford_ops.apply_to_PauliList(observable_small_angle_3e)

In [ ]:
observables_small_angle = {k:v for k,v in zip(['3a', '3b', '3c', '3d', '3e'], observables_small_angle_5steps + [observable_small_angle_3d, observable_small_angle_3e])}
observables_large_angle = {k:v for k,v in zip(['3a', '3b', '3c', '3d', '3e'], observables_large_angle_5steps + [observable_large_angle_3d, observable_large_angle_3e])}
ops_small_angle = {k:v for k,v in zip(['3a', '3b', '3c', '3d', '3e'], [ops_small_angle_5steps]*3 + [ops_small_angle_3d, ops_small_angle_3e])}
ops_large_angle = {k:v for k,v in zip(['3a', '3b', '3c', '3d', '3e'], [ops_large_angle_5steps]*3 + [ops_large_angle_3d, ops_large_angle_3e])}

with open('clifford_transforms.pkl', 'wb') as f:
    pickle.dump([ops_small_angle, ops_large_angle, observables_small_angle, observables_large_angle], f)

### Import Clifford transforms if the code above is commented out.

In [ ]:
with open('clifford_transforms.pkl', 'rb') as f:
    ops_small_angle, ops_large_angle, observables_small_angle, observables_large_angle = pickle.load(f)

### From Figure 3a

In [ ]:
nsteps = 5
coeffs_list = np.array([i*np.pi/64 for i in range(17)])
exp = []
for i, theta in enumerate(coeffs_list):
    if i<9:
        observable = observables_small_angle['3a']
        op_seq = OperatorSequence(ops_small_angle['3a'], [[theta]*nq]*nsteps).to_sparse_pauli_list()
    else:
        observable = observables_large_angle['3a']
        op_seq = OperatorSequence(ops_large_angle['3a'], [[theta-np.pi/4]*nq]*nsteps).to_sparse_pauli_list()
    e = np.real(Simulation.evolve_pauli_sum(observable, op_seq, threshold=0.001, progress_bar=False))/nq
    exp.append(e)
    print(i, e)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5,3.8))
ax.plot(coeffs_list*2/np.pi, exp)
ax.set_xlim(0, 1/2)
ax.set_xlabel(r'$\theta / \pi$')
plt.show()

### From Figure 3b

In [ ]:
nsteps = 5
coeffs_list = np.array([i*np.pi/64 for i in range(17)])
exp = []
for i, theta in enumerate(coeffs_list):
    if i<9:
        observable = observables_small_angle['3b']
        op_seq = OperatorSequence(ops_small_angle['3b'], [[theta]*nq]*nsteps).to_sparse_pauli_list()
    else:
        observable = observables_large_angle['3b']
        op_seq = OperatorSequence(ops_large_angle['3b'], [[theta-np.pi/4]*nq]*nsteps).to_sparse_pauli_list()
    e = np.real(Simulation.evolve_pauli_sum(observable, op_seq, threshold=0.00015, progress_bar=False))
    exp.append(e)
    print(i, e)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5,3.8))
ax.plot(coeffs_list*2/np.pi, exp)
ax.set_xlim(0, 1/2)
ax.set_xlabel(r'$\theta / \pi$')
plt.show()

### From Figure 3c

In [ ]:
nsteps = 5
coeffs_list = np.array([i*np.pi/64 for i in range(17)])
exp = []
for i, theta in enumerate(coeffs_list):
    if i<9:
        observable = observables_small_angle['3c']
        op_seq = OperatorSequence(ops_small_angle['3c'], [[theta]*nq]*nsteps).to_sparse_pauli_list()
    else:
        observable = observables_large_angle['3c']
        op_seq = OperatorSequence(ops_large_angle['3c'], [[theta-np.pi/4]*nq]*nsteps).to_sparse_pauli_list()
    e = np.real(Simulation.evolve_pauli_sum(observable, op_seq, threshold=0.00035, progress_bar=False))
    exp.append(e)
    print(i, e)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5,3.8))
ax.plot(coeffs_list*2/np.pi, -1 * np.array(exp))
ax.set_xlim(0, 1/2)
ax.set_xlabel(r'$\theta / \pi$')
plt.show()

### From Figure 3d

In [ ]:
nsteps = 6
coeffs_list = np.array([i*np.pi/64 for i in range(17)])
exp = []
for i, theta in enumerate(coeffs_list):
    if i<9:
        observable = observables_small_angle['3d']
        op_seq = OperatorSequence(ops_small_angle['3d'], [[theta]*nq]*nsteps).to_sparse_pauli_list()
    else:
        observable = observables_large_angle['3d']
        op_seq = OperatorSequence(ops_large_angle['3d'], [[theta-np.pi/4]*nq]*nsteps).to_sparse_pauli_list()
    e = np.real(Simulation.evolve_pauli_sum(observable, op_seq, threshold=0.00035, progress_bar=False))
    exp.append(e)
    print(i, e)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5,3.8))
ax.plot(coeffs_list*2/np.pi, -1 * np.array(exp))
ax.set_xlim(0, 1/2)
ax.set_xlabel(r'$\theta / \pi$')
plt.show()

### From Figure 3e

In [ ]:
nsteps = 20
coeffs_list = np.array([i*np.pi/64 for i in range(17)])
exp = []
for i, theta in enumerate(coeffs_list):
    if i<9:
        observable = observables_small_angle['3e']
        op_seq = OperatorSequence(ops_small_angle['3e'], [[theta]*nq]*nsteps).to_sparse_pauli_list()
    else:
        observable = observables_large_angle['3e']
        op_seq = OperatorSequence(ops_large_angle['3e'], [[theta-np.pi/4]*nq]*nsteps).to_sparse_pauli_list()
    e = np.real(Simulation.evolve_pauli_sum(observable, op_seq, threshold=0.0008, progress_bar=False))
    exp.append(e)
    print(i, e)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5,3.8))
ax.plot(coeffs_list*2/np.pi, 1 * np.array(exp))
ax.set_xlim(0, 1/2)
ax.set_xlabel(r'$\theta / \pi$')
plt.show()